<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase4/blob/main/Fase4_Tech_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconhecimento Facial

Desafio numero 4: analisando rostos, sentimentos e atividades.

## Instalar Dependências

In [2]:
# Instalar dependências necessárias
!pip install opencv-python opencv-python-headless matplotlib gdown deepface mediapipe fpdf face_recognition dlib

## Importar bibliotecas

In [3]:
# Importar bibliotecas
import gdown
import os
import cv2
import numpy as np
import time
import base64
from deepface import DeepFace
import face_recognition
import pandas as pd
import mediapipe as mp
from fpdf import FPDF

## Download do vídeo

In [3]:
# Baixar o vídeo diretamente do Google Drive
VIDEO_ID = "1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne"  # ID do arquivo no Google Drive
VIDEO_PATH = "/content/video.mp4"
gdown.download(f"https://drive.google.com/uc?id={VIDEO_ID}", VIDEO_PATH, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne
To: /content/video.mp4
100%|██████████| 38.3M/38.3M [00:00<00:00, 43.8MB/s]


'/content/video.mp4'

## Funções auxiliares

In [4]:
# Configuração do Mediapipe para detecção de atividades
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def detectar_faces(frame, face_cascade):
    # Convert to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    return frame, len(faces)

def analisar_emocao(frame):
    try:
        analysis = DeepFace.analyze(frame, actions=['emotion'], enforce_detection=False)
        return analysis[0]['dominant_emotion']
    except:
        return "desconhecido"

def detectar_atividade(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    return "Movimento detectado" if results.pose_landmarks else "Sem atividade"

def detectar_faces(frame, detector):
    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]

    # Detect faces using face_recognition
    face_locations = face_recognition.face_locations(rgb_frame)

    # Draw bounding boxes on detected faces
    for (top, right, bottom, left) in face_locations:
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

    return frame, len(face_locations)

def processar_video(video_path, output_path):
    # Load pre-trained face detection model
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    processed_frames = 0

    start_time = time.time() # Inicia a contagem do tempo

    print(f"Frames processados: {processed_frames}/{total_frames}")

    dados = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        frame, num_faces = detectar_faces(frame, face_cascade) # Pass None as detector is not used
        emocao = analisar_emocao(frame)
        atividade = detectar_atividade(frame)
        dados.append([frame_count, num_faces, emocao, atividade])
        out.write(frame)

        processed_frames += 1

        if processed_frames % 100 == 0:
            print(f"Frames processados: {processed_frames}/{total_frames}")

            end_time = time.time() # Finaliza a contagem do tempo
            total_time = end_time - start_time # Calcula o tempo total
            total_por_frame = total_time/processed_frames

            print(f"Tempo parcial de processamento: {total_time:.2f} segundos") # Imprime o tempo total
            print(f"Tempo parcial de processamento por frame: {total_por_frame:.2f} segundos") # Imprime o tempo total
            print(f"Tempo total de processamento previsto: {total_por_frame*total_frames} segundos") # Imprime o tempo total

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return dados

def gerar_relatorio(dados, relatorio_path):
    df = pd.DataFrame(dados, columns=['Frame', 'Rostos', 'Expressão', 'Atividade'])
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, "Relatório de Análise de Vídeo", ln=True, align='C')
    pdf.cell(200, 10, f"Total de Frames: {len(df)}", ln=True, align='L')
    pdf.cell(200, 10, f"Rostos Detectados: {df['Rostos'].sum()}", ln=True, align='L')
    pdf.output(relatorio_path)

## Fluxo principal

Execução das rotinas de processamento, usando as bibliotecas OpenCV, DLib, Mediapipe e Deepface. No final, gera um PDF.

In [5]:
# Processar vídeo com OpenCV, DLib, Mediapipe, Deepface
def main():
    video_path = "/content/video.mp4"
    output_path = "results/output_video.avi"
    relatorio_path = "results/relatorio.pdf"

    start_time = time.time() # Inicia a contagem do tempo

    # Create the 'results' directory if it doesn't exist
    os.makedirs(os.path.dirname(relatorio_path), exist_ok=True)

    dados = processar_video(video_path, output_path)
    gerar_relatorio(dados, relatorio_path)
    print("Processamento concluído! Relatório gerado.")

    end_time = time.time() # Finaliza a contagem do tempo
    total_time = end_time - start_time # Calcula o tempo total

    print(f"Tempo total de processamento: {total_time:.2f} segundos") # Imprime o tempo total

# Executar o pipeline
main()

Frames processados: 0/3326
Frames processados: 100/3326
Tempo parcial de processamento: 132.17 segundos
Tempo parcial de processamento por frame: 1.32 segundos
Tempo total de processamento previsto: 4395.852535586357 segundos
Frames processados: 200/3326
Tempo parcial de processamento: 257.56 segundos
Tempo parcial de processamento por frame: 1.29 segundos
Tempo total de processamento previsto: 4283.2843025922775 segundos
Frames processados: 300/3326
Tempo parcial de processamento: 410.48 segundos
Tempo parcial de processamento por frame: 1.37 segundos
Tempo total de processamento previsto: 4550.811364569664 segundos
Frames processados: 400/3326
Tempo parcial de processamento: 538.59 segundos
Tempo parcial de processamento por frame: 1.35 segundos
Tempo total de processamento previsto: 4478.411351009607 segundos
Frames processados: 500/3326
Tempo parcial de processamento: 638.76 segundos
Tempo parcial de processamento por frame: 1.28 segundos
Tempo total de processamento previsto: 4249